In [1]:
#Expand notebook to take full screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

#Jupyter magic to notify when a cell finishes execution with %%notify command -- does not work with Jupyterlab
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

###
import sys
sys.path.insert(0,'../src/')

%load_ext autoreload
%autoreload 2

#%pdb

<IPython.core.display.Javascript object>

In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.nn.modules import Module
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import copy
import pickle
import model_archs

from utils_plot import show_sws_weights, show_weights, print_dims, prune_plot, draw_sws_graphs, joint_plot, plot_data, show_sws_weights_log
from utils_model import test_accuracy, train_epoch, retrain_sws_epoch, model_prune, get_weight_penalty, layer_accuracy
from utils_misc import trueAfterN, logsumexp, root_dir, model_load_dir, get_ab, get_sparsity
from utils_sws import GaussianMixturePrior, special_flatten, KL, compute_responsibilies, merger, sws_prune, sws_prune_l2, sws_prune_copy
from mnist_loader import search_train_data, search_retrain_data, search_validation_data, train_data, test_data, batch_size
from extract_targets import get_targets
from retrain_layer import init_retrain_layer
from retrain_model import retrain_model

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle

retraining_epochs = 50

/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
! ls ../models/

fashionmnist_LeNet_300_100_100_full.m
fashionmnist_LeNet_300_100_100_full_targets
fashionmnist_SWSModel_100_full.m
fashionmnist_SWSModel_100_full_targets
mnist_LeNet_300_100_100_full.m
mnist_LeNet_300_100_100_full_targets
mnist_LeNet_300_100_100_search.m
mnist_LeNet_300_100_100_search_targets
mnist_SWSModel_100_full.m
mnist_SWSModel_100_full_targets
mnist_SWSModel_100_search.m
mnist_SWSModel_100_search_targets


In [5]:
test_data_full = Variable(test_data(fetch='data')).cuda()
test_labels_full = Variable(test_data(fetch='labels')).cuda()
model = torch.load("../models/mnist_LeNet_300_100_100_full.m").cuda()
print (test_accuracy(test_data_full, test_labels_full, model)[0])
model = torch.load("../models/mnist_SWSModel_100_full.m").cuda()
print (test_accuracy(test_data_full, test_labels_full, model)[0])

/anaconda/envs/py35/lib/python3.5/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'model_archs.LeNet_300_100' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


98.33
98.99


In [6]:
test_data_full = Variable(test_data(fetch='data', dset='fashionmnist')).cuda()
test_labels_full = Variable(test_data(fetch='labels', dset='fashionmnist')).cuda()
model = torch.load("../models/fashionmnist_LeNet_300_100_100_full.m").cuda()
print (test_accuracy(test_data_full, test_labels_full, model)[0])
model = torch.load("../models/fashionmnist_SWSModel_100_full.m").cuda()
print (test_accuracy(test_data_full, test_labels_full, model)[0])

/anaconda/envs/py35/lib/python3.5/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'model_archs.LeNet_300_100' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


89.63
89.91
